# Walk-Forward Backtest Report

This notebook loads the optional walk-forward report and visualizes per-horizon metrics.

## Notebook Guide

- **Purpose:** Visualize walk-forward backtest metrics.
- **Inputs:** reports/walk_forward_report.json
- **Outputs:** backtest plots.
- **Run:** Execute cells top‑to‑bottom. If a file is missing, run the earlier pipeline notebook first.


In [ ]:
import sys, subprocess
from pathlib import Path

print('Python:', sys.executable)
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
subprocess.run(['pip', 'install', '-e', str(repo_root)], check=True)


In [ ]:
import json
from pathlib import Path

report_path = Path(repo_root) / 'reports' / 'walk_forward_report.json'
print('Report path:', report_path)
if not report_path.exists():
    raise FileNotFoundError('walk_forward_report.json not found. Run training with backtest enabled.')

report = json.loads(report_path.read_text())
list(report.get('targets', {}).keys())


In [ ]:
import pandas as pd
import plotly.express as px

# Example: plot per-horizon RMSE for load_mw
metrics = report['targets']['load_mw']['gbm']['per_horizon']
rows = pd.DataFrame([
    {'horizon': int(k), 'rmse': v['rmse'], 'mae': v['mae'], 'smape': v['smape']}
    for k, v in metrics.items()
]).sort_values('horizon')

fig = px.line(rows, x='horizon', y=['rmse','mae','smape'], title='Load MW - GBM per-horizon metrics')
fig


In [ ]:
# Compare models for a target (RMSE)

def plot_compare(target='load_mw'):
    out = []
    for model in ['gbm', 'lstm', 'tcn']:
        if model not in report['targets'][target]:
            continue
        per = report['targets'][target][model]['per_horizon']
        for k, v in per.items():
            out.append({'model': model, 'horizon': int(k), 'rmse': v['rmse']})
    df = pd.DataFrame(out).sort_values(['model','horizon'])
    fig = px.line(df, x='horizon', y='rmse', color='model', title=f'{target} - RMSE by horizon')
    return fig

plot_compare('load_mw')


## Visual Sanity Checks

These plots provide a quick visual validation of recent behavior.

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
features_path = repo_root / 'data' / 'processed' / 'features.parquet'
if not features_path.exists():
    print('features.parquet not found. Run the feature pipeline first.')
else:
    df_viz = pd.read_parquet(features_path).sort_values('timestamp')
    if {'load_mw','wind_mw','solar_mw'}.issubset(df_viz.columns):
        recent = df_viz.tail(7 * 24)
        fig, ax = plt.subplots(3, 1, figsize=(12, 7), sharex=True)
        recent.plot(x='timestamp', y='load_mw', ax=ax[0], color='#1f77b4', title='Load (last 7 days)')
        recent.plot(x='timestamp', y='wind_mw', ax=ax[1], color='#2ca02c', title='Wind (last 7 days)')
        recent.plot(x='timestamp', y='solar_mw', ax=ax[2], color='#ff7f0e', title='Solar (last 7 days)')
        plt.tight_layout()
    else:
        print('Expected columns not found in features.parquet')
